In [41]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [42]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [43]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [44]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 18589.71it/s]


In [45]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 133.94it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [46]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [47]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [48]:
data = kernelclass.calc_kernel_mtxel()

In [49]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.6f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.478812-0.000000j | -0.029943+0.114178j | -0.070840+0.023855j | -0.069334-0.124695j | -0.102824-0.157271j | 0.016159+0.055476j | 0.023062-0.090879j | 0.000498+0.000065j
-0.029943-0.114178j | 0.216560+0.000000j | 0.023217+0.035262j | -0.013489+0.012930j | 0.027856-0.030818j | -0.015734+0.009451j | -0.100181+0.000795j | -0.002107-0.015270j
-0.070840-0.023855j | 0.023217-0.035262j | 0.216558+0.000000j | 0.003350+0.018209j | -0.010606-0.040923j | 0.004145-0.100101j | 0.010076-0.015572j | 0.000492+0.000236j
-0.069334+0.124695j | -0.013489-0.012930j | 0.003350-0.018209j | 0.242285-0.000000j | 0.099929-0.007170j | 0.006418+0.001458j | -0.004717-0.004452j | 0.050097-0.069127j
-0.102824+0.157271j | 0.027856+0.030818j | -0.010606+0.040923j | 0.099929+0.007170j | 0.216560+0.000000j | -0.017545-0.005331j | 0.012645+0.013767j | -0.009655+0.011491j
0.016159-0.055476j | -0.015734-0.009451j | 0.004145+0.100101j | 0.006418-0.001458j | -0.017545+0.005331j | 0.242346+0.000000j | -0.00

In [50]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.6f}")
    print(" | ".join(row_data))

1.000000-0.000000j | -0.050161+0.191307j | -0.113259+0.038148j | -0.119545-0.219404j | -0.179902-0.275188j | 0.028757+0.098114j | 0.038396-0.161090j | -0.000001-0.000000j
-0.050160-0.191313j | 1.000000+0.000000j | 0.044814+0.068058j | -0.243175+0.230883j | 0.053179-0.058835j | -0.282548+0.168881j | -0.030992+0.001391j | -0.000242-0.001754j
-0.113255-0.038140j | 0.044808-0.068063j | 1.000000-0.000000j | 0.059442+0.326677j | -0.020486-0.079059j | 0.001145-0.030978j | 0.179413-0.280033j | 0.000166+0.000079j
-0.123287+0.217328j | -0.240968-0.233186j | 0.060722-0.326442j | 1.000000-0.000000j | 0.030968-0.001559j | -0.042553-0.010903j | 0.031799+0.030240j | 0.014255-0.022259j
-0.179906+0.275184j | 0.053185+0.058837j | -0.020488+0.079056j | 0.030873+0.002878j | 1.000000+0.000000j | -0.315077-0.094531j | 0.228639+0.245554j | -0.001213+0.001443j
0.028428-0.098212j | -0.281817-0.170101j | 0.001420+0.030969j | -0.043087+0.008548j | -0.314413+0.096719j | 1.000000+0.000000j | 0.038188+0.021706j | -

In [51]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.6f}")
    print(" | ".join(row_data))

0.000000+0.000000j | -0.001637+0.006215j | -0.033643+0.011335j | 0.000939+0.001564j | -0.002884-0.004433j | -0.008854-0.030458j | -0.001788+0.005863j | -0.000501-0.000061j
-0.001630-0.006218j | 0.000000+0.000000j | 0.001529+0.002321j | -0.074256+0.070895j | -0.004186+0.004633j | -0.234892+0.140795j | -0.000594+0.000070j | 0.000128+0.000924j
-0.033641-0.011334j | 0.001529-0.002321j | 0.000000+0.000000j | -0.009812-0.056503j | -0.003401-0.013120j | -0.000036-0.003948j | 0.099764-0.153989j | 0.000597+0.000285j
0.000838-0.001620j | -0.073974-0.071189j | -0.010940+0.056295j | 0.000000+0.000000j | 0.000148-0.000032j | -0.006915-0.001412j | 0.001446+0.001485j | -0.010293+0.014328j
-0.002887+0.004431j | -0.004187-0.004632j | -0.003399+0.013120j | 0.000151-0.000011j | 0.000000+0.000000j | 0.090039+0.027446j | 0.072226+0.077994j | -0.000163+0.000194j
-0.008970+0.030424j | -0.234632-0.141222j | 0.000362+0.003931j | -0.006847+0.001714j | 0.090088-0.027280j | 0.000000+0.000000j | -0.000251-0.000046

In [52]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.6f}")
    print(" | ".join(row_data))

0.317954+0.000001j | -0.069654+0.265886j | -0.119995+0.040363j | -0.195908-0.349948j | -0.312321-0.477754j | 0.033496+0.113361j | 0.055035-0.213470j | 0.006986+0.000845j
-0.069684-0.265868j | 0.124536-0.000001j | 0.170757+0.259334j | -0.233811+0.225991j | 0.138481-0.153207j | -0.446454+0.270043j | -0.064171-0.001104j | -0.012140-0.087979j
-0.120007-0.040354j | 0.170755-0.259334j | 0.115235-0.000001j | 0.064602+0.344788j | -0.071078-0.274257j | 0.003062-0.065913j | 0.279260-0.428555j | 0.009154+0.004370j
-0.193866+0.351082j | -0.235681-0.224035j | 0.062344-0.345201j | 0.015061-0.000059j | 0.053376-0.004471j | 0.086149+0.021600j | -0.015798-0.014870j | 0.031792-0.040276j
-0.312357+0.477733j | 0.138472+0.153207j | -0.071068+0.274260j | 0.053467+0.003194j | 0.155066+0.000002j | -0.589978-0.181463j | 0.347449+0.381136j | -0.033546+0.039928j
0.032626-0.113615j | -0.448113-0.267274j | 0.002398+0.065936j | 0.087028-0.017733j | -0.591210+0.177398j | 0.026862+0.000862j | -0.112437-0.066271j | -0

In [53]:
l_epsinv = epsilon.l_epsinv
eps0mat = l_epsinv[0]

# for row in range(0, eps0mat.shape[0]):
#     for col in range(0, eps0mat.shape[1]):
#         print(f"eps0mat[{row}, {col}] = {eps0mat[row, col]}")


In [54]:
l_epsinv_sorted = kernelclass.l_epsinv
eps0mat_sorted = l_epsinv_sorted[0]

# for row in range(0, eps0mat_sorted.shape[0]):
#     for col in range(0, eps0mat_sorted.shape[1]):
#         print(f"eps0mat_sorted[{row}, {col}] = {eps0mat_sorted[row, col]}")